In [4]:
!pip install kaggle

In [ ]:
#upload the credentials of the kaggle account
from google.colab import files
files.upload()

In [6]:
# before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d corrieaar/apartment-rental-offers-in-germany

 87% 81.0M/93.0M [00:00<00:00, 178MB/s]
100% 93.0M/93.0M [00:00<00:00, 169MB/s]


In [8]:
!ls


apartment-rental-offers-in-germany.zip	kaggle.json  sample_data


In [9]:
# Extract the data
import zipfile
zip_ref = zipfile.ZipFile('apartment-rental-offers-in-germany.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [10]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [11]:
# Import the relevant files
df = pd.read_csv('/content/files/immo_data.csv')

In [12]:
df.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,...,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,...,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,...,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,...,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,...,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,...,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20


In [13]:
df.describe()

,serviceCharge,telekomHybridUploadSpeed,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,yearConstructedRange,...,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,livingSpaceRange,heatingCosts,lastRefurbish,electricityBasePrice,electricityKwhPrice
count,261941.000000,45020.0,268850.000000,267018.000000,235492.000000,2.283330e+05,211805.000000,2.688500e+05,93052.000000,211805.000000,...,268850.000000,162344.000000,217541.000000,171118.000000,268850.000000,268850.000000,85518.000000,80711.000000,46846.000000,46846.000000
mean,151.206113,10.0,9.791958,3.389001,28.804928,9.013315e+02,1966.400590,1.069697e+08,1.327634,3.714544,...,2.641261,114.749533,2.122405,3.572319,2.571542,3.070790,76.990866,2013.904536,89.113612,0.199769
std,308.295790,0.0,6.408399,1.964874,16.337151,3.323833e+04,46.992207,1.250093e+07,8.361403,2.738134,...,2.633440,61.653663,3.634934,6.375496,0.937594,1.407127,147.716278,10.963125,5.395805,0.009667
min,0.000000,10.0,0.000000,-12.330000,1.000000,0.000000e+00,1000.000000,2.887174e+07,0.000000,1.000000,...,1.000000,0.100000,-1.000000,0.000000,1.000000,1.000000,0.000000,1015.000000,71.430000,0.170500
25%,95.000000,10.0,6.000000,2.000000,10.000000,4.698000e+02,1950.000000,1.066910e+08,1.000000,1.000000,...,2.000000,79.000000,1.000000,2.000000,2.000000,2.000000,54.000000,2012.000000,90.760000,0.191500
50%,135.000000,10.0,9.000000,3.390000,40.000000,6.500000e+02,1973.000000,1.111584e+08,1.000000,3.000000,...,3.000000,107.000000,2.000000,3.000000,3.000000,3.000000,70.000000,2017.000000,90.760000,0.198500
75%,190.000000,10.0,13.000000,4.570000,40.000000,9.850000e+02,1996.000000,1.137688e+08,1.000000,5.000000,...,3.000000,140.300000,3.000000,4.000000,3.000000,4.000000,90.000000,2019.000000,90.760000,0.205500
max,146118.000000,10.0,121.000000,14.920000,100.000000,1.575154e+07,2090.000000,1.157117e+08,2241.000000,9.000000,...,999.990000,1996.000000,999.000000,999.000000,5.000000,7.000000,12613.000000,2919.000000,90.760000,0.227600


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268850 entries, 0 to 268849
Data columns (total 49 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   regio1                    268850 non-null  object 
 1   serviceCharge             261941 non-null  float64
 2   heatingType               223994 non-null  object 
 3   telekomTvOffer            236231 non-null  object 
 4   telekomHybridUploadSpeed  45020 non-null   float64
 5   newlyConst                268850 non-null  bool   
 6   balcony                   268850 non-null  bool   
 7   picturecount              268850 non-null  int64  
 8   pricetrend                267018 non-null  float64
 9   telekomUploadSpeed        235492 non-null  float64
 10  totalRent                 228333 non-null  float64
 11  yearConstructed           211805 non-null  float64
 12  scoutId                   268850 non-null  int64  
 13  noParkSpaces              93052 non-null   f

1

In [15]:
new_df=df[['serviceCharge','heatingType','telekomUploadSpeed','totalRent']]
new_df.head()

,serviceCharge,heatingType,telekomUploadSpeed,totalRent
0,245.00,central_heating,10.0,840.0
1,134.00,self_contained_central_heating,10.0,NaN
2,255.00,floor_heating,2.4,1300.0
3,58.15,district_heating,40.0,NaN
4,138.00,self_contained_central_heating,NaN,903.0


In [16]:
new_df.isnull().sum()

serviceCharge          6909
heatingType           44856
telekomUploadSpeed    33358
totalRent             40517
dtype: int64

In [17]:
df.dropna(subset=['totalRent'],inplace=True)
new_df.fillna(new_df._get_numeric_data().mean(),inplace = True)
new_df['heatingType'].fillna(new_df['heatingType'].value_counts().head(1).index[0],inplace = True)
new_df.head()

,serviceCharge,heatingType,telekomUploadSpeed,totalRent
0,245.00,central_heating,10.000000,840.000000
1,134.00,self_contained_central_heating,10.000000,901.331522
2,255.00,floor_heating,2.400000,1300.000000
3,58.15,district_heating,40.000000,901.331522
4,138.00,self_contained_central_heating,28.804928,903.000000


In [18]:

new_df.isnull().sum()

serviceCharge         0
heatingType           0
telekomUploadSpeed    0
totalRent             0
dtype: int64

In [19]:
new_df.head()

,serviceCharge,heatingType,telekomUploadSpeed,totalRent
0,245.00,central_heating,10.000000,840.000000
1,134.00,self_contained_central_heating,10.000000,901.331522
2,255.00,floor_heating,2.400000,1300.000000
3,58.15,district_heating,40.000000,901.331522
4,138.00,self_contained_central_heating,28.804928,903.000000


In [20]:
new_df = pd.get_dummies(new_df, columns = ['heatingType'])
new_df.head()

,serviceCharge,telekomUploadSpeed,totalRent,heatingType_central_heating,heatingType_combined_heat_and_power_plant,heatingType_district_heating,heatingType_electric_heating,heatingType_floor_heating,heatingType_gas_heating,heatingType_heat_pump,heatingType_night_storage_heater,heatingType_oil_heating,heatingType_self_contained_central_heating,heatingType_solar_heating,heatingType_stove_heating,heatingType_wood_pellet_heating
0,245.00,10.000000,840.000000,1,0,0,0,0,0,0,0,0,0,0,0,0
1,134.00,10.000000,901.331522,0,0,0,0,0,0,0,0,0,1,0,0,0
2,255.00,2.400000,1300.000000,0,0,0,0,1,0,0,0,0,0,0,0,0
3,58.15,40.000000,901.331522,0,0,1,0,0,0,0,0,0,0,0,0,0
4,138.00,28.804928,903.000000,0,0,0,0,0,0,0,0,0,1,0,0,0


In [21]:
x = new_df.drop(['totalRent'],axis=1).values
y = new_df['totalRent'].values

In [22]:
sc = StandardScaler()
x = sc.fit_transform(x)
Xtrain, Xtest, Ytrain, Ytest = train_test_split( x, y, test_size = 0.3,random_state=42)
def mse(x,y,w,b):
    cost = np.sum((((x.dot(w) + b) - y) ** 2) / (len(y)))
    return cost

def GradientDescent(x, y, w, b, learning_rate, epochs):
    cost_list = [0] * epochs
   
    for epoch in range(epochs):
        z = x.dot(w) + b
        loss = z - y
        
        weight_gradient = x.T.dot(loss) / len(y)
        bias_gradient = np.sum(loss) / len(y)
        
        w = w - learning_rate*weight_gradient
        b = b - learning_rate*bias_gradient
  
        cost = mse(x, y, w, b)
        cost_list[epoch] = cost
        
        if (epoch%(epochs/10)==0):
            print("mse :",cost)
        
    return w, b, cost_list
w, b, c= GradientDescent(Xtrain, Ytrain, np.zeros(Xtrain.shape[1]), 0, 0.002,epochs=15000)

mse : 1341149903.1302326
mse : 1339225053.6997817
mse : 1339198787.0463421
mse : 1339188733.977819
mse : 1339184089.236574
mse : 1339181940.895434
mse : 1339180947.2132235
mse : 1339180487.6007586
mse : 1339180275.014064
mse : 1339180176.6853487


In [23]:
def predict(X, w, b):
    return X.dot(w) + b
y_pred = predict(Xtest, w, b)

In [24]:
mse = mean_squared_error(Ytest, y_pred)
print(mse)


3154787.5153122097


2

In [25]:
lr_model = LinearRegression()
lr_model.fit(Xtrain, Ytrain)
lr_preds = lr_model.predict(Xtest)

mean_squared_error(Ytest, lr_preds)

3240131.3034362276

3

In [26]:
df = df.drop(columns=df.columns[((df.isna().sum()/len(df)) > 0.50)])
df.dropna(subset=['totalRent'],inplace=True)
df = df.drop(df[df['totalRent'] == 0.0].index)
df = df.drop(df[df['livingSpace'] == 0.0].index)
df = df.drop(columns=['description','street','facilities','livingSpaceRange','scoutId','streetPlain','houseNumber','date','geo_bln'])
df.fillna(df._get_numeric_data().mean(),inplace = True)
for cols in df.columns:
    if df[cols].dtype == 'int64' or df[cols].dtype == 'float64':
        upper_range = df[cols].mean() + 3 * df[cols].std()
        lower_range = df[cols].mean() - 3 * df[cols].std()
        indexs = df[(df[cols] > upper_range) | (df[cols] < lower_range)].index
        df = df.drop(indexs)
for cols in df.columns:
    if df[cols].dtype == 'object' or df[cols].dtype == 'bool':
        df[cols].fillna(df[cols].value_counts().head(1).index[0],inplace = True)
df = df.drop(columns=['firingTypes','geo_krs','regio2','regio3'])
cat_cols = []
for cols in df.columns:
    if df[cols].dtype == 'object' or df[cols].dtype == 'bool':
        cat_cols.append(cols)
df = pd.concat([df, pd.get_dummies(df[cat_cols])], axis=1)
df = df.drop(columns=cat_cols)

In [27]:
X = df.drop('totalRent',axis=1)
Y=df['totalRent']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [28]:
reg = Ridge().fit(X_train, y_train)
y_pred=reg.predict(X_test)
print(mean_absolute_error(y_test, y_pred))

27.39234027083926


In [29]:
reg = Lasso().fit(X_train, y_train)
y_pred=reg.predict(X_test)
print(mean_absolute_error(y_test, y_pred))

28.436480247714194


2.2.1.1

In [33]:
sc = StandardScaler()
x = sc.fit_transform(X)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, Y, test_size = 0.3,random_state=42)
def abse(x,y,w,b):
    cost = np.sum(np.absolute(((x.dot(w) + b) - y)) / (2*len(y)))
    return cost

def GradientDescent(x, y, w, b, learning_rate, epochs):
    cost_list = [0] * epochs
   
    for epoch in range(epochs):
        z = x.dot(w) + b
        loss = z - y
        
        weight_gradient = x.T.dot(loss) / len(y)
        bias_gradient = np.sum(loss) / len(y)
        
        w = w - learning_rate*weight_gradient
        b = b - learning_rate*bias_gradient
  
        cost = abse(x, y, w, b)
        cost_list[epoch] = cost
        
        if (epoch%(epochs/10)==0):
            print("Absolute Error :",cost)
        
    return w, b, cost_list
w, b, c= GradientDescent(Xtrain, Ytrain, np.zeros(Xtrain.shape[1]), 0, 0.002,epochs=15000)

Absolute Error : 380.3247937099075
Absolute Error : 31.314993003435756
Absolute Error : 22.879008870895532
Absolute Error : 20.739727166187382
Absolute Error : 19.18661218539636
Absolute Error : 17.996808570940562
Absolute Error : 17.086127649166883
Absolute Error : 16.393343910831696
Absolute Error : 15.869114970629832
Absolute Error : 15.476157294030676


In [34]:
def predict(X, w, b):
    return X.dot(w) + b
y_pred = predict(Xtest, w, b)

In [35]:
Absolute_Error =mean_absolute_error(Ytest, y_pred)
print(Absolute_Error)

29.162655360158997
